In [1]:
import dlib
import cv2
import numpy as np

# Dlib's face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('path/to/shape_predictor_68_face_landmarks.dat')

# Step 1: Face extraction and key-point detection
def extract_face_and_keypoints(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = detector(gray)
    
    if len(faces) == 0:
        return None, None
    
    # Extract facial key-points using landmark predictor
    landmarks = predictor(gray, faces[0])
    
    return faces[0], landmarks

# Example usage
image_path = 'path/to/image.jpg'
face, landmarks = extract_face_and_keypoints(image_path)

# Step 2: Face alignment
def align_face(image, landmarks):
    # Extracting key-point coordinates
    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    right_eye = (landmarks.part(45).x, landmarks.part(45).y)
    nose = (landmarks.part(30).x, landmarks.part(30).y)
    
    # Calculating center points
    glabella_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
    forehead_center = ((landmarks.part(0).x + landmarks.part(2).x) // 2, (landmarks.part(19).y + landmarks.part(24).y) // 2)
    
    # Constructing a triangle for angle calculation
    triangle = np.array([forehead_center, nose, glabella_center], dtype='float32')
    
    # Computing rotation angle
    angle = np.degrees(np.arctan2(glabella_center[1] - nose[1], glabella_center[0] - nose[0]))
    
    # Rotating the image
    rotated_face = cv2.warpAffine(image, cv2.getRotationMatrix2D(glabella_center, angle, 1.0), (image.shape[1], image.shape[0]))
    
    return rotated_face

# Example usage
aligned_face = align_face(face, landmarks)

# Step 3: Resizing
def resize_image(image, size=(224, 224)):
    return cv2.resize(image, size)

# Example usage
resized_face = resize_image(aligned_face)

# Displaying the results
cv2.imshow('Original Face', face)
cv2.imshow('Aligned and Resized Face', resized_face)
cv2.waitKey(0)
cv2.destroyAllWindows()

RuntimeError: Unable to open path/to/shape_predictor_68_face_landmarks.dat